In [ ]:
import keyring
import getpass
import functools
import itertools
import tempfile
import io

import numpy as np
import matplotlib.pyplot as plt

import imageio
import skimage.transform

import pydicom

import segments

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
from pymedphys._experimental.autosegmentation import pipeline, mask

In [ ]:
EXPANSION = 4

In [ ]:
segments_api_key = keyring.get_password('segments-ai', 'api-key')

if not segments_api_key:
    segments_api_key = getpass.getpass()
    keyring.set_password('segments-ai', 'api-key', segments_api_key)

In [ ]:
client = segments.SegmentsClient(segments_api_key)
dataset_name = 'SimonBiggs/AnimalContours' # Name of a dataset you've created on Segments.ai

In [ ]:
(
    data_path_root,
    structure_set_paths,
    ct_image_paths,
    ct_uid_to_structure_uid,
    structure_uid_to_ct_uids,
    names_map,
    structure_names_by_ct_uid,
    structure_names_by_structure_set_uid,
    uid_to_url,
    hash_path,
) = pipeline.get_dataset_metadata()

In [ ]:
ct_uids = ct_image_paths.keys()

In [ ]:
def get_dcm_ct_from_uid(ct_uid):
    ct_path = ct_image_paths[ct_uid]
    dcm_ct = pydicom.read_file(ct_path, force=True)

    dcm_ct.file_meta.TransferSyntaxUID = pydicom.uid.ImplicitVRLittleEndian

    return dcm_ct

In [ ]:
for ct_uid in ct_uids:
    pipeline.download_uid(data_path_root, ct_uid, uid_to_url, hash_path)
    dcm_ct = get_dcm_ct_from_uid(ct_uid)
    
    HU = dcm_ct.pixel_array * dcm_ct.RescaleSlope + dcm_ct.RescaleIntercept
    
    new_ct_size = np.array(np.shape(HU)) * EXPANSION    
    expanded = skimage.transform.resize(HU, new_ct_size, mode='edge', preserve_range=True, anti_aliasing=False)
    uint16_image = ((expanded + 1024) / 4096 * 2**16).astype(np.uint16)
    png_file = io.BytesIO()
    imageio.imsave(png_file, uint16_image, format='png', prefer_uint8=False)
    
    sample_name = f"{ct_uid}.png"
    asset = client.upload_asset(png_file, filename=sample_name)
    image_url = asset["url"]
    
    attributes = {
        "image": { "url": image_url }
    }
    result = client.add_sample(dataset_name, sample_name, attributes)